# Preprocessing the Datasets

## Merging datasets

### 📊 Dataset: tracks and artists

Start by loading the datasets:

In [1]:
import pandas as pd

raw_artists = pd.read_csv("../data/raw/spotify_artists_cleaned.csv")
raw_tracks = pd.read_csv("../data/raw/spotify_tracks.csv")

Lets see how they look like.
- Artists:

In [2]:
raw_artists.head()

,artist_id,total_followers,genres,name,artist_popularity
0,6LqNN22kT3074XbTVUrhzX,8311262,"['dance pop', 'pop']",Kesha,81
1,3nFkdlSjzX9mRTtwJOzDYB,9579702,"['east coast hip hop', 'gangster rap', 'hip ho...",JAY-Z,85
2,1HY2Jd0NmPuamShAr6KMms,32915877,"['art pop', 'dance pop', 'pop']",Lady Gaga,92
3,5tKXB9uuebKE34yowVaU3C,1009481,"['dance pop', 'pop rap', 'post-teen pop']",Iyaz,66
4,2dnXvETqd6QKCJn821Orxg,9693,[],Norhye,33


- Tracks:

In [3]:
raw_tracks.head()

,index,track_id,title,artists,explicit,track_popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,1,0HPD5WQqrq7wPWR7P7Dw1i,TiK ToK,['Kesha'],False,83,0.09910,0.755,199693,0.837,0.000000,2,0.2890,-2.718,0,0.1420,120.028,4,0.714
1,2,2igwFfvr1OAGX9SKDCPBwO,Empire State Of Mind,"['JAY-Z', 'Alicia Keys']",True,80,0.02950,0.491,276920,0.956,0.000000,11,0.4600,-1.538,1,0.3920,173.585,4,0.811
2,3,0SiywuOBRcynK0uKGWdCnn,Bad Romance,['Lady Gaga'],True,82,0.00314,0.695,294573,0.921,0.000053,0,0.0842,-3.752,1,0.0363,119.007,4,0.714
3,4,5vlEg2fT4cFWAqU5QptIpQ,Replay,['Iyaz'],False,69,0.17300,0.706,182307,0.751,0.000000,9,0.1680,-6.323,1,0.0708,91.031,4,0.195
4,5,3DamFFqW32WihKkTVlwTYQ,Fireflies,['Owl City'],False,76,0.02750,0.512,228347,0.662,0.000000,3,0.1180,-6.797,1,0.0439,180.114,4,0.472


Okay, artists are looking good for now. Lets remove the extra index column.

In [4]:
raw_tracks = raw_tracks.drop(columns=["index"])
raw_tracks.head()

,track_id,title,artists,explicit,track_popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0HPD5WQqrq7wPWR7P7Dw1i,TiK ToK,['Kesha'],False,83,0.09910,0.755,199693,0.837,0.000000,2,0.2890,-2.718,0,0.1420,120.028,4,0.714
1,2igwFfvr1OAGX9SKDCPBwO,Empire State Of Mind,"['JAY-Z', 'Alicia Keys']",True,80,0.02950,0.491,276920,0.956,0.000000,11,0.4600,-1.538,1,0.3920,173.585,4,0.811
2,0SiywuOBRcynK0uKGWdCnn,Bad Romance,['Lady Gaga'],True,82,0.00314,0.695,294573,0.921,0.000053,0,0.0842,-3.752,1,0.0363,119.007,4,0.714
3,5vlEg2fT4cFWAqU5QptIpQ,Replay,['Iyaz'],False,69,0.17300,0.706,182307,0.751,0.000000,9,0.1680,-6.323,1,0.0708,91.031,4,0.195
4,3DamFFqW32WihKkTVlwTYQ,Fireflies,['Owl City'],False,76,0.02750,0.512,228347,0.662,0.000000,3,0.1180,-6.797,1,0.0439,180.114,4,0.472


Nice, now lets tackle the first problem. The artist dataset gives us valuable artist data such as genre. We want this data but how do we merge this when a track can have multiple artists? We have 3 solutions here:
1. **Expanding Rows (Denormalize):** Each track-artist combination becomes one row. This is a good solution if you want analyse the artsits impact.

2. **Aggregate Artist Features:** When predicting track success, you want one row per track. E.g. the sum, mean, max for numerical and a list of all values for categorical features.

3. **Main Artist Only:** This would only make sense if artists have small influence on the tracks popularity. 

For our solution we want to use aggregate.

### 🛠️ Aggregate Artsits: 

Here is a step-by-step solution:
1. Write a function that goes through the *artists* aggregates the numerical and categorical features.
2. Apply it to the track dataset. 
3. Merge with original dataframe. 

Start with the function:

In [5]:
import ast


def aggregate_artist_data(artist_names: list[str]):
    # get all the artist names
    # convert string interpretation of the input as a python array.
    artist_names = [name.lower() for name in ast.literal_eval(artist_names)]
    # dataframe
    matched_artists = raw_artists[
        raw_artists["name"].str.lower().isin([name.lower() for name in artist_names])
    ]

    # if no artists are found, return None for all values
    if matched_artists.empty:
        pd.Series(
            {
                "artist_avg_popularity": 0,
                "artist_avg_followers": 0,
                "artist_combined_genres": [],
            }
        )

    # aggregate data
    avg_popularity = matched_artists["artist_popularity"].mean()
    avg_followers = matched_artists["total_followers"].mean()
    combined_genres = set(
        genre for sublist in matched_artists["genres"] for genre in eval(sublist)
    )

    # return all values in a dictionary
    return pd.Series(
        {
            "artist_avg_popularity": avg_popularity,
            "artist_avg_followers": avg_followers,
            "artist_combined_genres": list(combined_genres),
        }
    )

Now lets see if this works by applying it to the artists in a track: 

In [6]:
aggregate_artist_data = raw_tracks["artists"].apply(aggregate_artist_data)
aggregate_artist_data.head()

,artist_avg_popularity,artist_avg_followers,artist_combined_genres
0,81.0,8311262.0,"[dance pop, pop]"
1,83.0,10909491.0,"[gangster rap, pop rap, neo soul, pop, r&b, hi..."
2,92.0,32915877.0,"[dance pop, art pop, pop]"
3,66.0,1009481.0,"[dance pop, pop rap, post-teen pop]"
4,70.0,2430925.0,"[indietronica, pop]"


Awesome! Now we only need to concatenate this to our tracks dataset:

In [7]:
tracks_with_aggregated_artist_data = pd.concat(
    [raw_tracks, aggregate_artist_data], axis=1
)
tracks_with_aggregated_artist_data.head()

,track_id,title,artists,explicit,track_popularity,acousticness,danceability,duration_ms,energy,instrumentalness,...,liveness,loudness,mode,speechiness,tempo,time_signature,valence,artist_avg_popularity,artist_avg_followers,artist_combined_genres
0,0HPD5WQqrq7wPWR7P7Dw1i,TiK ToK,['Kesha'],False,83,0.09910,0.755,199693,0.837,0.000000,...,0.2890,-2.718,0,0.1420,120.028,4,0.714,81.0,8311262.0,"[dance pop, pop]"
1,2igwFfvr1OAGX9SKDCPBwO,Empire State Of Mind,"['JAY-Z', 'Alicia Keys']",True,80,0.02950,0.491,276920,0.956,0.000000,...,0.4600,-1.538,1,0.3920,173.585,4,0.811,83.0,10909491.0,"[gangster rap, pop rap, neo soul, pop, r&b, hi..."
2,0SiywuOBRcynK0uKGWdCnn,Bad Romance,['Lady Gaga'],True,82,0.00314,0.695,294573,0.921,0.000053,...,0.0842,-3.752,1,0.0363,119.007,4,0.714,92.0,32915877.0,"[dance pop, art pop, pop]"
3,5vlEg2fT4cFWAqU5QptIpQ,Replay,['Iyaz'],False,69,0.17300,0.706,182307,0.751,0.000000,...,0.1680,-6.323,1,0.0708,91.031,4,0.195,66.0,1009481.0,"[dance pop, pop rap, post-teen pop]"
4,3DamFFqW32WihKkTVlwTYQ,Fireflies,['Owl City'],False,76,0.02750,0.512,228347,0.662,0.000000,...,0.1180,-6.797,1,0.0439,180.114,4,0.472,70.0,2430925.0,"[indietronica, pop]"


Great, now we have combined the track data with the aggregation of our artist data. Lets save this dataset before moving on.

In [8]:
tracks_with_aggregated_artist_data.to_csv(
    "../data/raw/spotify_tracks_with_aggregated_artist_data.csv", index=False
)

### 📊 Datasets: Billboard Top 100 and Spotify Tracks with Aggregated Artist Data

Lets look at the data again, but this time lets frst see what the song with most weeks on charts was.

In [9]:
raw_billboard_data = pd.read_csv("../data/raw/billboard_top_100_2010-2024.csv")
raw_billboard_data.sort_values("weeks on chart", ascending=False).head()

,index,position,title,artist,last week,peak position,weeks on chart,week of
66820,66821,21,Heat Waves,Glass Animals,20,1,91,2022-10-22
66719,66720,20,Heat Waves,Glass Animals,19,1,90,2022-10-15
60919,60920,20,Blinding Lights,The Weeknd,21,1,90,2021-09-04
60820,60821,21,Blinding Lights,The Weeknd,18,1,89,2021-08-28
66618,66619,19,Heat Waves,Glass Animals,17,1,89,2022-10-08


Interesting... but, there are multiple issues here.

Lets tackle the logic part first. For each song we have many duplicate entries. For what we want to do, we only need the overall popularity measured by billboard. What we need is a popularity measure that tells us how popular a song was based on the weeks on chart and the peak position. 

Lets drop the index column again, too and format the column names coherently with the other datasets.

In [10]:
import numpy as np

raw_billboard_data = raw_billboard_data.drop(columns=["index"])
raw_billboard_data.rename(
    columns={
        "artist": "artists",
        "last week": "last_week",
        "peak position": "peak_position",
        "weeks on chart": "weeks_on_chart",
        "week of": "week_of",
    },
    inplace=True,
)

Okay now lets write some functions to calculate billboards popularity and transforms the data to only have a single entry.

But first, lets clean up a little.

In [11]:
# Converts '-' and non-numeric to NaN
raw_billboard_data["weeks_on_chart"] = pd.to_numeric(
    raw_billboard_data["weeks_on_chart"], errors="coerce"
)
raw_billboard_data["peak_position"] = pd.to_numeric(
    raw_billboard_data["peak_position"], errors="coerce"
)
raw_billboard_data = raw_billboard_data.dropna(
    subset=["weeks_on_chart", "peak_position"]
)

# Make sure the 'week of' col is in a valid date format
raw_billboard_data["week_of"] = pd.to_datetime(raw_billboard_data["week_of"])

# This ensures we only keep the most recent entry for each title
raw_billboard_data_last_entry = (
    raw_billboard_data.sort_values("week_of").groupby("title").tail(1)
)

raw_billboard_data_last_entry.sort_values("weeks_on_chart", ascending=False).head()

,position,title,artists,last_week,peak_position,weeks_on_chart,week_of
66820,21,Heat Waves,Glass Animals,20,1,91,2022-10-22
60919,20,Blinding Lights,The Weeknd,21,1,90,2021-09-04
22748,49,Radioactive,Imagine Dragons,48,3,87,2014-05-10
22044,45,Sail,AWOLNATION,41,17,79,2014-03-22
64417,18,Levitating,Dua Lipa,21,2,77,2022-05-07


Nice, now we only have the last entries with the minimum peak position and highest weeks on chart. Let's now calculate the popularity score.

### 📈 Billboard Popularity Score

For the popularity score we need two things: 
- based on min peak position and max weeks on charts
- ensure no disproportionate domination of tracks with extreme values.

This can be achieved with `(101 - peak_position) * np.log1p(weeks_on_chart)`

And we also want to eliminate duplicates by only leaving the entry with `min. peak position` and `max. weeks on chart`

In [12]:
def compute_popularity(row):
    peak_position = row["peak_position"]
    weeks_on_chart = row["weeks_on_chart"]

    return (101 - peak_position) * np.log1p(weeks_on_chart)


def transform_billboard_dataset(data):
    # compute the popularity score
    data_agg = (
        data.groupby("title")
        .agg(
            {
                "artists": "first",  # Keep the first artist for each title
                "peak_position": "min",  # Track with the best (lowest) peak position
                "weeks_on_chart": "max",  # Track's total charting weeks
                "week_of": "max",  # Keep the most recent week
            }
        )
        .reset_index()
    )

    data_agg["billboard_popularity_score"] = data_agg.apply(compute_popularity, axis=1)

    return data_agg[["title", "artists", "billboard_popularity_score", "week_of"]]


billboard_data = transform_billboard_dataset(raw_billboard_data_last_entry)
billboard_data.head()

,title,artists,billboard_popularity_score,week_of
0,#Beautiful,Mariah Carey Featuring Miguel,243.656348,2013-09-07
1,#SELFIE,The Chainsmokers,211.217065,2014-05-24
2,#thatPOWER,will.i.am Featuring Justin Bieber,237.989921,2013-07-20
3,$ave Dat Money,Lil Dicky Featuring Fetty Wap & Rich Homie Quan,89.871968,2016-03-19
4,'98 Braves,Morgan Wallen,153.878674,2023-04-29


Looking cleannn, but oh no what's this?

**🚨 Major Issue:** There is a quadzillion ways to write the name of an artist or a feature description.

We need some kind of fuzzy string matching and some regular expressions to solve this.

In [13]:
from rapidfuzz import process
import re

### 💬 Fuzzy String Matching
Lets start by writing a `clean` function that formats the artist column in a way that is useful for us.

In [14]:
def clean(name: str):
    name = name.lower()
    name = re.sub(r"[^a-z0-9 ]", " ", name)  # remove anything not letter/number/space
    name = re.sub(r"\s+", " ", name)  # collapse spaces
    return name.strip()


billboard_data["artists"] = billboard_data["artists"].apply(clean)
billboard_data["title"] = billboard_data["title"].apply(clean)

tracks_with_aggregated_artist_data["artists"] = tracks_with_aggregated_artist_data[
    "artists"
].apply(
    lambda x: " ".join([clean(name) for name in ast.literal_eval(x)])
)  # convert to string for use in fuzzy matching
tracks_with_aggregated_artist_data["title"] = tracks_with_aggregated_artist_data[
    "title"
].apply(clean)

tracks_with_aggregated_artist_data.sort_values("title").head()

,track_id,title,artists,explicit,track_popularity,acousticness,danceability,duration_ms,energy,instrumentalness,...,liveness,loudness,mode,speechiness,tempo,time_signature,valence,artist_avg_popularity,artist_avg_followers,artist_combined_genres
2030,4jtyUzZm9WLc2AdaJ1dso7,0 to 100 the catch up,drake,True,66,0.737000,0.559,275227,0.716,0.000000,...,0.252,-3.964,0,0.3900,176.618,4,0.504,96.000000,9.360711e+07,"[canadian pop, pop rap, canadian hip hop, hip ..."
7141,582YxhgpJEY0aEHQtVQFhO,02 02 99,that mexican ot,True,58,0.161000,0.848,162162,0.630,0.000000,...,0.130,-6.543,1,0.2660,148.092,4,0.602,73.000000,1.022206e+06,[texas latin rap]
4423,226le7T3p82reYWzsi9Hsz,1 2 many,luke combs brooks dunn,False,73,0.039700,0.540,180907,0.821,0.000000,...,0.423,-3.789,1,0.0873,148.798,4,0.685,80.500000,7.381128e+06,"[country road, contemporary country, country]"
4045,2wOXxtHZgRkkrkEbKLzzqs,1 5,21 savage,True,56,0.000244,0.885,148656,0.520,0.000001,...,0.454,-8.353,1,0.3590,84.022,4,0.306,89.000000,2.071186e+07,"[atl hip hop, rap, hip hop]"
3176,5tz69p7tJuGPeMGwNTxYuV,1 800 273 8255,logic alessia cara khalid,True,73,0.569000,0.620,250173,0.574,0.000000,...,0.190,-7.788,0,0.0479,100.023,4,0.357,78.333333,1.148472e+07,"[canadian pop, canadian contemporary r&b, pop ..."


Okay, this looks acceptable. Last thing we need to do is use the fuzzy matching score on the title and artist for the datasets and choose the value with the highest score combination to be the correct row. 

In [15]:
def fuzzy_merge(billboard, tracks):
    # Join the artist and title columns in a string for tracks
    tracks["title_and_artists"] = tracks["title"] + " " + tracks["artists"]
    billboard["title_and_artists"] = billboard["title"] + " " + billboard["artists"]

    # Get the best match for each title_and_artists in billboard
    tracks["best_match"] = [
        process.extractOne(query, billboard["title_and_artists"])[0]
        for query in tracks["title_and_artists"]
    ]

    # Merge the dataframes on the best match
    merged = pd.merge(
        tracks,
        billboard,
        left_on="best_match",
        right_on="title_and_artists",
        suffixes=("_tracks", "_billboard"),
    )

    # Drop the extra columns
    merged = merged.drop(
        columns=[
            "best_match",
            "title_billboard",
            "artists_billboard",
            "title_and_artists_billboard",
            "title_and_artists_tracks",
        ]
    )
    return merged


# Perform fuzzy merge using both title and artist
billboard_and_tracks = fuzzy_merge(
    billboard_data,
    tracks_with_aggregated_artist_data,
)

billboard_and_tracks.head()

,track_id,title_tracks,artists_tracks,explicit,track_popularity,acousticness,danceability,duration_ms,energy,instrumentalness,...,mode,speechiness,tempo,time_signature,valence,artist_avg_popularity,artist_avg_followers,artist_combined_genres,billboard_popularity_score,week_of
0,0HPD5WQqrq7wPWR7P7Dw1i,tik tok,kesha,False,83,0.09910,0.755,199693,0.837,0.000000,...,0,0.1420,120.028,4,0.714,81.0,8311262.0,"[dance pop, pop]",366.356165,2010-07-10
1,2igwFfvr1OAGX9SKDCPBwO,empire state of mind,jay z alicia keys,True,80,0.02950,0.491,276920,0.956,0.000000,...,1,0.3920,173.585,4,0.811,83.0,10909491.0,"[gangster rap, pop rap, neo soul, pop, r&b, hi...",31.884770,2010-01-02
2,0SiywuOBRcynK0uKGWdCnn,bad romance,lady gaga,True,82,0.00314,0.695,294573,0.921,0.000053,...,1,0.0363,119.007,4,0.714,92.0,32915877.0,"[dance pop, art pop, pop]",354.768375,2017-02-25
3,5vlEg2fT4cFWAqU5QptIpQ,replay,iyaz,False,69,0.17300,0.706,182307,0.751,0.000000,...,1,0.0708,91.031,4,0.195,66.0,1009481.0,"[dance pop, pop rap, post-teen pop]",82.001919,2010-10-09
4,3DamFFqW32WihKkTVlwTYQ,fireflies,owl city,False,76,0.02750,0.512,228347,0.662,0.000000,...,1,0.0439,180.114,4,0.472,70.0,2430925.0,"[indietronica, pop]",346.573590,2010-04-03


Okay, we are almost done. There are still some values we dont want. Lets get rid of these now and save this step.

In [ ]:
billboard_and_tracks = billboard_and_tracks.dropna(
    subset=[
        "billboard_popularity_score",
        "artist_avg_popularity",
        "artist_avg_followers",
    ]
)

billboard_and_tracks.head()

billboard_and_tracks.to_csv("../data/raw/billboard_and_tracks_with_aggregated_artists_unnormalised.csv", index=False)

Great work! Now to our last data preprocessing step.

### 🧹 Numerical Feature Normalisation

Before get into it we need to look at our data. For which values does a normalisation make sense? 

In [18]:
raw_data = pd.read_csv(
    "../data/raw/billboard_and_tracks_with_aggregated_artists_unnormalised.csv"
)
raw_data.head()

,track_id,title_tracks,artists_tracks,explicit,track_popularity,acousticness,danceability,duration_ms,energy,instrumentalness,...,mode,speechiness,tempo,time_signature,valence,artist_avg_popularity,artist_avg_followers,artist_combined_genres,billboard_popularity_score,week_of
0,0HPD5WQqrq7wPWR7P7Dw1i,tik tok,kesha,False,83,0.09910,0.755,199693,0.837,0.000000,...,0,0.1420,120.028,4,0.714,81.0,8311262.0,"['dance pop', 'pop']",366.356165,2010-07-10
1,2igwFfvr1OAGX9SKDCPBwO,empire state of mind,jay z alicia keys,True,80,0.02950,0.491,276920,0.956,0.000000,...,1,0.3920,173.585,4,0.811,83.0,10909491.0,"['gangster rap', 'pop rap', 'neo soul', 'pop',...",31.884770,2010-01-02
2,0SiywuOBRcynK0uKGWdCnn,bad romance,lady gaga,True,82,0.00314,0.695,294573,0.921,0.000053,...,1,0.0363,119.007,4,0.714,92.0,32915877.0,"['dance pop', 'art pop', 'pop']",354.768375,2017-02-25
3,5vlEg2fT4cFWAqU5QptIpQ,replay,iyaz,False,69,0.17300,0.706,182307,0.751,0.000000,...,1,0.0708,91.031,4,0.195,66.0,1009481.0,"['dance pop', 'pop rap', 'post-teen pop']",82.001919,2010-10-09
4,3DamFFqW32WihKkTVlwTYQ,fireflies,owl city,False,76,0.02750,0.512,228347,0.662,0.000000,...,1,0.0439,180.114,4,0.472,70.0,2430925.0,"['indietronica', 'pop']",346.573590,2010-04-03


Alright, here is a list of features we want to normalise:
1. track_popularity
2. duration_ms
3. key
4. loudness
5. tempo
6. time_signature
7. artist_avg_popularity
8. artist_avg_followers
9. billboard_popularity_score

The rest of the data is either binary, categorical or already normalised. 

In [21]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
features_to_normalise = [
    "track_popularity",
    "duration_ms",
    "key",
    "loudness",
    "tempo",
    "time_signature",
    "artist_avg_popularity",
    "artist_avg_followers",
    "billboard_popularity_score",
]
raw_data[features_to_normalise] = scaler.fit_transform(
    raw_data[features_to_normalise]
)
raw_data.head()


,track_id,title_tracks,artists_tracks,explicit,track_popularity,acousticness,danceability,duration_ms,energy,instrumentalness,...,mode,speechiness,tempo,time_signature,valence,artist_avg_popularity,artist_avg_followers,artist_combined_genres,billboard_popularity_score,week_of
0,0HPD5WQqrq7wPWR7P7Dw1i,tik tok,kesha,False,0.83,0.09910,0.755,0.231409,0.837,0.000000,...,0,0.1420,0.431105,0.75,0.714,0.797872,0.065666,"['dance pop', 'pop']",0.809910,2010-07-10
1,2igwFfvr1OAGX9SKDCPBwO,empire state of mind,jay z alicia keys,True,0.80,0.02950,0.491,0.341263,0.956,0.000000,...,1,0.3920,0.754885,0.75,0.811,0.819149,0.086194,"['gangster rap', 'pop rap', 'neo soul', 'pop',...",0.069087,2010-01-02
2,0SiywuOBRcynK0uKGWdCnn,bad romance,lady gaga,True,0.82,0.00314,0.695,0.366374,0.921,0.000053,...,1,0.0363,0.424933,0.75,0.714,0.914894,0.260062,"['dance pop', 'art pop', 'pop']",0.784245,2017-02-25
3,5vlEg2fT4cFWAqU5QptIpQ,replay,iyaz,False,0.69,0.17300,0.706,0.206678,0.751,0.000000,...,1,0.0708,0.255804,0.75,0.195,0.638298,0.007976,"['dance pop', 'pop rap', 'post-teen pop']",0.180092,2010-10-09
4,3DamFFqW32WihKkTVlwTYQ,fireflies,owl city,False,0.76,0.02750,0.512,0.272169,0.662,0.000000,...,1,0.0439,0.794356,0.75,0.472,0.680851,0.019206,"['indietronica', 'pop']",0.766094,2010-04-03


Perfect! Lets save this dataset to our processed folder and we are done.

In [22]:
raw_data.to_csv(
    "../data/processed/billboard_and_tracks_with_aggregated_artists_normalised.csv",
    index=False,
)